In [1]:
import os
import shutil
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_genai import ChatGoogleGenerativeAI

/home/thibaud.garnier@Digital-Grenoble.local/Documents/RAG_proj/RAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Chargement des PDF   OCR

In [2]:
pip install pi-heif


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.4 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [11]:
pdf_folder = "./documents"

all_docs = []

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0,
    max_tokens=200, 
    timeout=None,
    max_retries=2
)

# Parcours de tous les fichiers PDF du dossier
for file_name in os.listdir(pdf_folder):
    if file_name.lower().endswith(".pdf"):
        file_path = os.path.join(pdf_folder, file_name)
        print(f"📄 Chargement du fichier : {file_name}")
        
        try:
            loader = PyPDFLoader(file_path)
            docs = loader.load()

            content_excerpt = docs


            # Préparer le prompt pour générer un tag
            messages = [
                ("system", "Tu es un assistant qui lit un document et génère deux ou trois mots-clés pour indiqué l'objet concerné par le document."),
                ("human", f"Voici le texte : {content_excerpt}\nDonne seulement un ou deux mots-clés séparés par des virgules.")
            ]

            ai_msg = llm.invoke(messages)
            for doc in docs:
                doc.metadata["source"] = file_name
                doc.metadata["tag"] = ai_msg.content.strip()
                
                
            all_docs.extend(docs)
        except Exception as e:
            print(f"⚠️ Erreur lors du chargement de {file_name}: {e}")

print(f"\n✅ {len(all_docs)} pages chargées depuis {len(os.listdir(pdf_folder))} fichiers PDF.")


📄 Chargement du fichier : airfryer.pdf
📄 Chargement du fichier : SV25-FR.pdf

✅ 39 pages chargées depuis 2 fichiers PDF.


In [12]:
for doc in docs:
    print(doc.metadata)

{'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2022-06-03T01:45:29+05:30', 'moddate': '2022-06-03T01:47:20+05:30', 'trapped': '/False', 'author': 'Unknown User Name', 'source': 'SV25-FR.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1', 'tag': 'Aspirateur, Entretien'}
{'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2022-06-03T01:45:29+05:30', 'moddate': '2022-06-03T01:47:20+05:30', 'trapped': '/False', 'author': 'Unknown User Name', 'source': 'SV25-FR.pdf', 'total_pages': 16, 'page': 1, 'page_label': '2', 'tag': 'Aspirateur, Entretien'}
{'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2022-06-03T01:45:29+05:30', 'moddate': '2022-06-03T01:47:20+05:30', 'trapped': '/False', 'author': 'Unknown User Name', 'source': 'SV25-FR.pdf', 'total_pages': 16, 'page': 2, 'page_label': '3', 'tag': 'Aspirateur, Entretien'}
{'producer': 'Ad

Split des docs  CHUNKING

In [13]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True)

all_splits = splitter.split_documents(all_docs)

In [14]:
all_splits

[Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 16.3 (Windows)', 'creationdate': '2021-07-23T16:01:18+08:00', 'author': 'Freek Bosgraaf', 'moddate': '2021-07-23T16:01:21+08:00', 'subject': 'adaptations cover 11/05/2007', 'title': 'Modèle-A5', 'trapped': '/False', 'source': 'airfryer.pdf', 'total_pages': 23, 'page': 0, 'page_label': '1', 'tag': "Airfryer, Mode d'emploi", 'start_index': 0}, page_content="Mode d'emploi\nHD927X"),
 Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 16.3 (Windows)', 'creationdate': '2021-07-23T16:01:18+08:00', 'author': 'Freek Bosgraaf', 'moddate': '2021-07-23T16:01:21+08:00', 'subject': 'adaptations cover 11/05/2007', 'title': 'Modèle-A5', 'trapped': '/False', 'source': 'airfryer.pdf', 'total_pages': 23, 'page': 1, 'page_label': '2', 'tag': "Airfryer, Mode d'emploi", 'start_index': 0}, page_content="2\nFRANÇAIS\nTABLE DES MATIÈRES\nImportant   3\nIntroduction   7\nDescription générale   

Chargement de l'embeddings

In [15]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")



Création de la base vectorielle avec Chroma  EMBEDDINGS

In [ ]:
persist_dir = "chroma_db"

# Supprimer l'ancienne base si elle existe
if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)
    print(f"Ancienne base '{persist_dir}' supprimée ✅")


ValueError: Expected metadata value to be a str, int, float, bool, SparseVector, or None, got ['Airfryer', "Mode d'emploi"] which is a list in upsert.

Try filtering complex metadata from the document using langchain_community.vectorstores.utils.filter_complex_metadata.

In [16]:

# Créer la base vectorielle locale
vectordb = Chroma.from_documents(
    documents=all_splits,
    embedding=embeddings,
    persist_directory=persist_dir
)

# Sauvegarde de la base
vectordb.persist()
print("Base vectorielle créée et sauvegardée ✅")

Base vectorielle créée et sauvegardée ✅


/tmp/ipykernel_27332/2872438712.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


Recherche sémantique dans la db

In [ ]:
vectordbload = Chroma(persist_directory="chroma_db", embedding_function=embeddings)
query = "Combien de temps pour recharger l'aspirateur'' ?"
results = vectordbload.similarity_search(query, k=10)

keyword = "aspirateur"
filtered_results = [
    r for r in results 
    if keyword in r.metadata.get("tag", "").lower()
]

print("🔍 Résultats les plus pertinents :\n")
for i, res in enumerate(filtered_results, 1):
    print(f"--- Résultat {i} ---")
    print(res.page_content)
    print("Source:", res.metadata["source"])
    print()


Chargement de la clé API

In [ ]:
from dotenv import load_dotenv
# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

# Vérifier la présence de la clé
if not os.environ.get("GOOGLE_API_KEY"):
    raise ValueError("⚠️ GOOGLE_API_KEY manquante dans le fichier .env")

Appel au LLM

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)
messages = [
    (
        "system",
        "Tu es un assistant qui répond aux question d'un utilisateur sur la documentation technique d'un produit, utilise les information complémentaire envoyé par le rag.",
    ),
    ("human", "Quel est le temps de charge du dyson?."),
    ("assistant",f"voila les résultats du rag: {results}")
]
ai_msg = llm.invoke(messages)
ai_msg
print(ai_msg.content)



Une recharge complète de votre aspirateur Dyson nécessite environ **4,5 heures**.

Pendant la charge, les LED clignotent pour indiquer le niveau de batterie actuel. Elles s'éteignent une fois que l'appareil est complètement rechargé.
